所要時間：　プログラム実行時間とパラメータチューニングを含めて2時間です（コードのみだと30分）  
なお, データおよび課題の特徴から, 事前分析はしていません。  

参考URL：  
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html  
https://lightgbm.readthedocs.io/en/latest/Python-API.html  
https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst  

In [1]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from progressbar import ProgressBar

In [2]:
from bayes_opt import  Bayes_opt_lgb_v2

In [3]:
train = pd.read_csv("./dota2Dataset/dota2Train.csv", header=None)
test = pd.read_csv("./dota2Dataset/dota2Test.csv", header=None)
target_tr = train[0]
target_te = test[0]
target_tr[target_tr < 0] = 0
target_te[target_te < 0] = 0
train = train.drop([0], axis=1)
test = test.drop([0], axis=1)

In [4]:
for i in range(4, len(train.iloc[0,3:])):
    if len(np.unique(train[i].values)) == 1:
        train = train.drop([i], axis=1)
        test = test.drop([i], axis=1)
        print("feature ", i , "is dropped.")

feature  27 is dropped.
feature  111 is dropped.


In [5]:
for i in range(0, 3):
    train.iloc[:,i] = train.iloc[:,i].astype("category")
    test.iloc[:,i] = test.iloc[:,i].astype("category")

In [7]:
# params は探索するパラメータ
params = {
        'max_depth': [3, 10],
        'num_leaves' : [8, 64],
        'learning_rate' : [0.01, 0.1],
        'feature_fraction' : [0.6, 0.99],
        'bagging_fraction' : [0.6, 0.99],
        'bagging_freq': [1, 10],
        'min_data_in_leaf': [3, 25],
        'subsample': [0.6, 0.99],
        'colsample_bytree': [0.6, 0.99],
        'reg_lambda': [0.25, 15.0],
        'reg_alpha': [0.25, 15.0],  
}

#basic_paramsは固定パラメータ
basic_params = {
    'boosting_type' : 'gbdt',
    'objective' : 'binary', #回帰ならobjective = regression
    'metric' : {'binary_logloss'}, #回帰ならmseなど
    #'num_class': 3, # 回帰ならコメントアウト
    'verbose' : 0,
    'num_threads':15
}

bayes_lgb = Bayes_opt_lgb_v2(opt_param = params, basic_params = basic_params, dat_size = 30000)
opt_parameter = bayes_lgb.fit(train.values, target_tr.values, ep=50, y_amp=1000.0, maximize=False)


[10.   64.    0.1   0.99  0.99 10.   25.    0.99  0.99 15.   15.  ]
epoch: 0
parameter:  {'max_depth': 4, 'num_leaves': 44, 'learning_rate': 0.054513093658002776, 'feature_fraction': 0.7727758295985573, 'bagging_fraction': 0.9244454299987683, 'bagging_freq': 1, 'min_data_in_leaf': 20, 'subsample': 0.8765911238387747, 'colsample_bytree': 0.7035707894411178, 'reg_lambda': 4.138141978927047, 'reg_alpha': 2.4680735421110476, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 0, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.677422 + 0.000435692
[200]	cv_agg's binary_logloss: 0.67186 + 0.000609847
[300]	cv_agg's binary_logloss: 0.668726 + 0.000681906
[400]	cv_agg's binary_logloss: 0.666819 + 0.000808594
[500]	cv_agg's binary_logloss: 0.665483 + 0.000800415
[600]	cv_agg's binary_logloss: 0.664554 + 0.000822181
[700]	cv_agg's binary_logloss: 0.663895 + 0.000803183
[800]	cv_agg's binary_logloss: 0.663435 + 0.000862341
[900]	cv_agg's binary_logloss: 0.

[100]	cv_agg's binary_logloss: 0.669144 + 0.000786637
[200]	cv_agg's binary_logloss: 0.664834 + 0.00103372
[300]	cv_agg's binary_logloss: 0.663522 + 0.00111697
[400]	cv_agg's binary_logloss: 0.662988 + 0.00115179
[500]	cv_agg's binary_logloss: 0.662587 + 0.00121632
[600]	cv_agg's binary_logloss: 0.662258 + 0.00133461
evaluation:  0.662146308542305 y_point: 0.7625289972891247
selected: 28532

epoch: 10
parameter:  {'max_depth': 10, 'num_leaves': 12, 'learning_rate': 0.07918208972231792, 'feature_fraction': 0.7540853069878477, 'bagging_fraction': 0.9817880950137035, 'bagging_freq': 1, 'min_data_in_leaf': 23, 'subsample': 0.7649865324075028, 'colsample_bytree': 0.8653929092030455, 'reg_lambda': 3.1955098362759404, 'reg_alpha': 14.624527271974477, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 0, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.670498 + 0.000670002
[200]	cv_agg's binary_logloss: 0.66597 + 0.000768191
[300]	cv_agg's binary_loglos

[100]	cv_agg's binary_logloss: 0.668543 + 0.000837968
[200]	cv_agg's binary_logloss: 0.66446 + 0.00105069
[300]	cv_agg's binary_logloss: 0.6633 + 0.00109073
[400]	cv_agg's binary_logloss: 0.662673 + 0.00121805
[500]	cv_agg's binary_logloss: 0.662376 + 0.0011496
[600]	cv_agg's binary_logloss: 0.662161 + 0.00130428
evaluation:  0.6621364768815752 y_point: 0.7723606580190001
selected: 3490

epoch: 19
parameter:  {'max_depth': 10, 'num_leaves': 16, 'learning_rate': 0.09664706316735795, 'feature_fraction': 0.6312529768824275, 'bagging_fraction': 0.868889953947767, 'bagging_freq': 3, 'min_data_in_leaf': 22, 'subsample': 0.9322344489090866, 'colsample_bytree': 0.8061331958602276, 'reg_lambda': 3.3611564959760716, 'reg_alpha': 13.724752629322431, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 0, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.668242 + 0.000982058
[200]	cv_agg's binary_logloss: 0.664207 + 0.00115628
[300]	cv_agg's binary_logloss: 0.

[100]	cv_agg's binary_logloss: 0.667217 + 0.000994932
[200]	cv_agg's binary_logloss: 0.665204 + 0.00117891
evaluation:  0.6651330755219659 y_point: -2.224237982371702
selected: 15236

epoch: 28
parameter:  {'max_depth': 6, 'num_leaves': 18, 'learning_rate': 0.09516067930488066, 'feature_fraction': 0.8535649324267335, 'bagging_fraction': 0.689121671476373, 'bagging_freq': 1, 'min_data_in_leaf': 15, 'subsample': 0.6210734541076143, 'colsample_bytree': 0.8390432228159378, 'reg_lambda': 12.95694846793265, 'reg_alpha': 14.988016562088168, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 0, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.670245 + 0.000718168
[200]	cv_agg's binary_logloss: 0.665318 + 0.00110814
[300]	cv_agg's binary_logloss: 0.663272 + 0.00116511
[400]	cv_agg's binary_logloss: 0.662452 + 0.0012784
[500]	cv_agg's binary_logloss: 0.662096 + 0.0014981
evaluation:  0.6620079788987947 y_point: 0.9008586407994734
selected: 8551

epoch: 29

[100]	cv_agg's binary_logloss: 0.67351 + 0.000542591
[200]	cv_agg's binary_logloss: 0.667979 + 0.000847321
[300]	cv_agg's binary_logloss: 0.665248 + 0.00101384
[400]	cv_agg's binary_logloss: 0.663805 + 0.00112319
[500]	cv_agg's binary_logloss: 0.662927 + 0.00127282
[600]	cv_agg's binary_logloss: 0.662484 + 0.00133032
[700]	cv_agg's binary_logloss: 0.662221 + 0.00136325
evaluation:  0.6621056401832067 y_point: 0.8031973563874528
selected: 24230

epoch: 38
parameter:  {'max_depth': 10, 'num_leaves': 16, 'learning_rate': 0.0885350310315625, 'feature_fraction': 0.828327398233152, 'bagging_fraction': 0.6913972416655624, 'bagging_freq': 2, 'min_data_in_leaf': 25, 'subsample': 0.6300515428204614, 'colsample_bytree': 0.6856905368159849, 'reg_lambda': 1.7028617887575854, 'reg_alpha': 13.30232106618079, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 0, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.668346 + 0.000850975
[200]	cv_agg's binary_logloss:

[500]	cv_agg's binary_logloss: 0.662697 + 0.0013692
[600]	cv_agg's binary_logloss: 0.662383 + 0.00145673
evaluation:  0.6622827661949016 y_point: 0.6260713446926136
selected: 21527

epoch: 47
parameter:  {'max_depth': 9, 'num_leaves': 11, 'learning_rate': 0.06925237142437, 'feature_fraction': 0.9211009504512486, 'bagging_fraction': 0.9738786553232329, 'bagging_freq': 1, 'min_data_in_leaf': 21, 'subsample': 0.6905377138559078, 'colsample_bytree': 0.8719979682782533, 'reg_lambda': 7.6188723777376275, 'reg_alpha': 13.18288172250873, 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'}, 'verbose': 0, 'num_threads': 15}
[100]	cv_agg's binary_logloss: 0.671899 + 0.000658423
[200]	cv_agg's binary_logloss: 0.666994 + 0.000929044
[300]	cv_agg's binary_logloss: 0.664927 + 0.00107488
[400]	cv_agg's binary_logloss: 0.663972 + 0.00116138
[500]	cv_agg's binary_logloss: 0.663468 + 0.00122978
[600]	cv_agg's binary_logloss: 0.663103 + 0.00121093
[700]	cv_agg's binary_logloss: 0.

In [8]:
opt_parameter

{'max_depth': 10,
 'num_leaves': 12,
 'learning_rate': 0.0776588506764468,
 'feature_fraction': 0.6427029677550093,
 'bagging_fraction': 0.6048721007485673,
 'bagging_freq': 1,
 'min_data_in_leaf': 23,
 'subsample': 0.6723574929586689,
 'colsample_bytree': 0.921362407369235,
 'reg_lambda': 12.077002300894526,
 'reg_alpha': 12.53736396348507}

In [9]:
parameters = {**basic_params, **opt_parameter}

In [10]:
parameters["verbose"] = 0

In [16]:
for i in range(5):
    print ("ensamble ", i)
    X_train2, X_test2, y_train2, y_test2 = train_test_split(
                train.values, target_tr.values, test_size=0.1, random_state=1+i)

    lgb_train = lgb.Dataset(X_train2, y_train2.reshape(-1,))
    lgb_eval = lgb.Dataset(X_test2, y_test2.reshape(-1,), reference=lgb_train)
    model = lgb.train(parameters, lgb_train, valid_sets=lgb_eval, num_boost_round=1000, early_stopping_rounds=10, verbose_eval=False)
    if i == 0:
        result = model.predict(test.values, num_iteration=model.best_iteration)
    else:
        result += model.predict(test.values, num_iteration=model.best_iteration)
result /= 5

ensamble  0
ensamble  1
ensamble  2
ensamble  3
ensamble  4


In [17]:
result_c = (2*result).astype(np.int32) # 確率出力のため, 2倍して1未満かどうかでラベルに変更

In [18]:
print ("正答率：", np.sum(target_te.values == result_c), "/", len(target_te), " ", np.sum(target_te.values == result_c)/len(target_te))

正答率： 6126 / 10294   0.5951039440450748


In [19]:
from sklearn.metrics import accuracy_score
print ("Accuracy:",accuracy_score(target_te.values, result_c))

Accuracy: 0.5951039440450748
